In [2]:
#We are going to use mne to import tables
import os
from tqdm import tqdm #Library used to see the progress of our loop
import mne
import pandas as pd
import numpy as np

There is some information about the data in a file called `French_information_.xlsx` . We get that information first:

In [33]:
df_info = pd.read_excel("SEED_FRA/French_information_.xlsx")

In [34]:
n_trials = 20

trial_num = tuple(f"Trial-{i}" for i in range(n_trials+1))
start_second = df_info["Trial Start second"][:n_trials+1]
end_second = df_info["Trial End second"][:n_trials+1]

See which channels are used and see if the order is correct

In [1]:
#Reading a cnt
def read_cnt(_path, _name, _trial_num, _start_second, _end_second):
    """Reads a .cnt file and stores in a parquet file the value of each signal"""
    eeg_raw = mne.io.read_raw_cnt(_path, preload=False)
    useless_ch = ['M1', 'M2', 'VEO', 'HEO']
    eeg_raw.drop_channels(useless_ch)
    new_ch = eeg_raw.ch_names
    
    sample_freq = 1000
    
    for trial, start, end in tqdm(zip(_trial_num, _start_second, _end_second), desc="Processing", total=len(_trial_num)):
        
        start_sample = int(start * sample_freq)
        end_sample = int(end * sample_freq)

        # Read data in chunks and concatenate them
        chunk_size = 10000
        chunks = []
        for chunk_start in range(start_sample, end_sample, chunk_size):
            chunk_end = min(chunk_start + chunk_size, end_sample)
            chunk_data = eeg_raw.get_data(start=chunk_start, stop=chunk_end)
            chunks.append(chunk_data)
        
        data_name = np.concatenate(chunks, axis=1).T
        df = pd.DataFrame(data_name, columns=new_ch)
        new_file_name = f"{_name.rstrip('.cnt')}-signals.parquet"
        df.to_parquet(f"SEED_FRA_OUTPUT/{new_file_name}", compression='gzip')

In [37]:
#Gets the file names in the folder and selects just those with .cnt extension
file_names = os.listdir("SEED_FRA/EEG_raw")
cnt_names = tuple(filter(lambda s: ".cnt" in s, file_names))

In [51]:
for file_name in cnt_names:
    read_cnt(f"SEED_FRA/EEG_raw/{file_name}", file_name, trial_num, start_second, end_second)

C:\Users\marco\AppData\Local\Temp\ipykernel_33824\158349187.py:3: RuntimeWarning:   Could not parse meas date from the header. Setting to None.
  eeg_raw = mne.io.read_raw_cnt(_name, preload=False)
Processing: 100%|██████████████████████████████████████████████████████████████████████| 21/21 [01:44<00:00,  4.97s/it]
C:\Users\marco\AppData\Local\Temp\ipykernel_33824\158349187.py:3: RuntimeWarning:   Could not parse meas date from the header. Setting to None.
  eeg_raw = mne.io.read_raw_cnt(_name, preload=False)
Processing: 100%|██████████████████████████████████████████████████████████████████████| 21/21 [01:49<00:00,  5.20s/it]
C:\Users\marco\AppData\Local\Temp\ipykernel_33824\158349187.py:3: RuntimeWarning:   Could not parse meas date from the header. Setting to None.
  eeg_raw = mne.io.read_raw_cnt(_name, preload=False)
Processing: 100%|██████████████████████████████████████████████████████████████████████| 21/21 [01:50<00:00,  5.26s/it]
C:\Users\marco\AppData\Local\Temp\ipykernel_33